In [3]:
from elasticsearch import Elasticsearch

import shared.config as config
import shared.const as const
import pymongo
import numpy as np
import pandas as pd
import json
import re
import ast
import logging

from pymongo import ReplaceOne
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
#from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, ForeignKey, Float
from sqlalchemy.orm import relationship, Session, declarative_base
from sqlalchemy.sql import column, text
from sqlalchemy import select, func
from sqlalchemy.sql.expression import literal

logger = logging.getLogger()

In [12]:

Base = declarative_base()
index_name = "abr_test2"

class WasstraatModel(Base):
    __abstract__ = True

    herkomst = Column(Text)
    key = Column(Text)
    soort = Column(String(80))
    brondata = Column(Text)
    uuid = Column('_id', String)

class Project(WasstraatModel): # Inherit from Model for cannot use Abstract class Wasstraatmodel, for geo-package gives errors
    __tablename__ = 'Def_Project'

    primary_key = Column(Integer, primary_key=True, autoincrement=True)
    projectcd = Column(String(12), nullable=False, index=True)
    projectnaam = Column(String(1024), index=True)
    jaar = Column(Integer)
    toponiem = Column(String(1024))
    trefwoorden = Column(String(1024))
    #location = Column(Geometry('POINT', srid=4326), default=(52.00667, 4.35556)) # 52.00667, 4.35556.
    xcoor_rd = Column(Float)
    ycoor_rd = Column(Float)
    longitude = Column(Float)
    latitude = Column(Float)
    #artefacten = relationship("Artefact", back_populates="project")

    # Explicit defined for cannot use Abstract class Wasstraatmodel, for geo-package gives errors
    herkomst = Column(Text)
    soort = Column(String(80))
    brondata = Column(Text)
    uuid = Column('_id', String)
    key = Column(Text)

    @staticmethod
    def getDescription(p):
        return str(p.projectcd) + f' ({p.projectnaam if p.projectnaam and p.projectnaam != "" else "Zonder naam"})'

    def __repr__(self):
        return Project.getDescription(self)


engine = create_engine(config.SQLALCHEMY_DATABASE_URI)    
session = Session(engine)

#query = session.query(User).filter(User.name.like("%ed")).order_by(User.id)

q = session.query(Project).filter(Project.projectcd.like('DC%'))
result = q.all()

with engine.connect() as connection:
    insp = inspect(connection)
    tables = insp.get_columns('Def_Artefact')
    [print(table) for table in tables if table['name'] == 'artefactsoort']


session.query(Project).filter(Project.jaar > 1999).count()

{'name': 'artefactsoort', 'type': ENUM('Aardewerk', 'Dierlijk_Bot', 'Glas', 'Hout', 'Bouwaardewerk', 'Kleipijp', 'Leer', 'Menselijk_Bot', 'Metaal', 'Munt', 'Onbekend', 'Schelp', 'Steen', 'Textiel', name='discrartefactsoortenum'), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}


199

In [3]:
from sqlalchemy.dialects import postgresql

pkey_json = [{'primary_key': hit['_id'], 'score': hit['_score']} for hit in resp['hits']['hits']]
pkey_json = json.dumps(pkey_json) 
print(pkey_json)

engine = create_engine(config.SQLALCHEMY_DATABASE_URI)    
session = Session(engine)
result = session.execute(sql)

q = session.query(Project).filter(Project.projectcd.like('DC%'))
subquery = text(f"SELECT * FROM json_to_recordset('{pkey_json}')" + ' as x("primary_key" int, "score" float)')
subquery= subquery.columns(column('primary_key'), column('score'))
subquery = subquery.subquery()
q = q.join(subquery, Project.primary_key == subquery.c.primary_key).add_columns(subquery.c.score.label("fulltext_score")).order_by(subquery.c.score.desc())
print(str(q.statement.compile(dialect=postgresql.dialect())))

df = pd.DataFrame(q.all())
df

NameError: name 'resp' is not defined

In [80]:
es = Elasticsearch('http://elasticsearch:9200')
highlight = {"fields": {"*": {}}}
index = 'def_artefact'
fields=['*']
maxsize=50
filter_path=['hits.hits._id', 'hits.hits._score', 'hits.hits.highlight','hits.hits._source.artefactsoort','hits.total']

query={'query': {'bool': {'filter': [{'multi_match': {'query': 'Leer',
      'fields': ['artefactsoort']}}],
   'must': [{'simple_query_string': {'query': 'kind*', 'fields': ['*']}}]}},
 'from': 0,
 'size': 1000,
 'highlight': {'fields': {'*': {}}}}
query={'bool': {'filter': [{'multi_match': {'query': 'Leer', 'fields': ['artefactsoort']}}],
                'must': [{'simple_query_string': {'query': 'kind*', 'fields': ['*']}}]}}

   
#query={'simple_query_string': {'query': 'Kinderschoen', 'analyze_wildcard':True, "fields": ['*']}}
postfilter={"term": { "_source.artefactsoort": "Leer" }} 

resp = es.search(
        index=index,
        query=query,
        size=maxsize,
        highlight=highlight,
    filter_path=filter_path,
    #post_filter=postfilter
    
)
    

print("Got %d Hits:" % resp['hits']['total']['value'])
#for hit in resp['hits']['hits']:
#    print(hit)
resp['hits']['hits'][0]

Got 64 Hits:


{'_id': '84875',
 '_score': 3.0,
 '_source': {'artefactsoort': 'Leer'},
 'highlight': {'beschrijving': ['<em>kinderzool</em>, smal midden en hak'],
  'beschrijving.keyword': ['<em>kinderzool, smal midden en hak</em>'],
  'artefactsoort': ['<em>Leer</em>'],
  'zool': ['enkele zool van <em>kinderschoen</em>']}}

In [ ]:
resp = es.search(index='my_index', query={'simple_query_string': {'query': 'scond~'}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit)

In [ ]:
lst_skipindex = ['_id', 'herkomst', 'key', 'brondata']


def generate_docs(resultset, db_col_names, index_name):
    for row in resultset:
        doc = {
            "_index": index_name,
            "_id": int(row["primary_key"]),
            "doc": {col.lower(): str(row[col] if str(row[col]) != 'None' else '') for col in db_col_names}
        }
        doc['doc']['id'] = int(row["primary_key"])
        yield doc


def indexTable(table):
    logger.info(f"Starting indexing table {table} for fulltext indexing...")

    engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
    logger.info("Connecting to " + config.SQLALCHEMY_DATABASE_URI)
    es = Elasticsearch('http://elasticsearch:9200')
    
    index_name = table.lower()

    with engine.connect() as connection:
        connection = connection.execution_options( 
            isolation_level="SERIALIZABLE",
            postgresql_deferrable=True # Does not seem to work. Work imn progress 
        )
        with connection.begin():
            metadata = db.MetaData(bind=engine)
            db.MetaData.reflect(metadata)

            #get the table list
            dict_tables = metadata.tables.keys()
            lst_tables = [x for x in list(dict_tables) if x.startswith('Def_')]
            logger.info("Indexing all string fields for " + str(lst_tables))

            if table in lst_tables:
                insp = inspect(connection)
                db_cols = insp.get_columns(table)
                db_cols = [col for col in db_cols if (col['name'] == 'primary_key' or 'text' in str(col['type']).lower() or 'varchar' in str(col['type']).lower()) and col['name'] not in lst_skipindex]
                sql_col_names = ['"'+col['name']+ '"' for col in db_cols]
                db_col_names = [col['name'] for col in db_cols]
                
                sql = f'SELECT {", ".join(sql_col_names)} FROM public."{table}"'
                logger.info(f"Indexing index {index_name} with columns {db_col_names} ")
                rs = connection.execute(sql)
                
                logger.info(f"Indexing index {index_name} with columns {db_col_names} ")
                helpers.bulk(es, generate_docs(rs, db_col_names, index_name))
                logger.info(f"Indexed {es.count(index=index_name)} records")                
            else:
                logger.error(f"Trying to index table {table}, but table not available in {lst_tables}")

es = Elasticsearch('http://elasticsearch:9200')
indexTable('Def_Artefact')

In [ ]:
es.options(ignore_status=[400,404]).indices.delete(index='_all')

In [ ]:
from sqlalchemy import select, func
stmt = select(func.json_each('{"a":"foo", "b":"bar"}').table_valued("key", "value"))
print(stmt)

session.rollback()
result = session.execute(stmt)
df = pd.DataFrame(result)
df


In [ ]:
from sqlalchemy import select, func, column, Integer, Text
stmt = select(
    func.json_to_recordset(str(pkey_json)).table_valued(
        column("primary_key", Integer), column("score", Float),
    ).render_derived(name="t", with_types=True)
)
print(stmt)

#session.rollback()
#result = session.execute(stmt)
#df = pd.DataFrame(result)
#df
subquery = session.query(func.json_to_recordset(str(pkey_json)).\
                             table_valued(column("primary_key", Integer), column("score", Float)).\
                             render_derived(name="t", with_types=True)).\
                             subquery()

'''
sub_query = session.query(
    0.9 * func.sum(Revenue.total_revenue)
).select_from(
    Revenue
).filter(
    func.datediff(func.now(), Revenue.date) <= 7
).subquery()
'''

q = session.query(Project).filter(Project.projectcd.like('DC%'))
#subquery = text(f"SELECT * FROM json_to_recordset('{pkey_json}')" + ' as x("primary_key" int, "score" float)')
#subquery= subquery.columns(column('primary_key'), column('score'))
#subquery = subquery.subquery()
q = q.join(subquery, Project.primary_key == subquery.c.primary_key).add_columns(subquery.c.score.label("fulltext_score")).order_by(subquery.c.score.desc())
print(str(q.statement.compile(dialect=postgresql.dialect())))

df = pd.DataFrame(q.all())
df

In [ ]:
conversion_table = (
    (
        "is_relation_many_to_one",
        ['FilterRelationOneToManyEqual', 'FilterRelationOneToManyNotEqual'],
    ),
    (
        "is_relation_one_to_one",
        ['FilterRelationOneToManyEqual', 'FilterRelationOneToManyNotEqual'],
    ),
    (
        "is_text",
        [
            'FilterStartsWith',
            'FilterEndsWith',
        ],
    )
)
conversion_table = dict(conversion_table)
#conversion_table['is_text'] = conversion_table['is_text'] + ['Test']


#conversion_table = tuple([(k, v) for k, v in conversion_table.items()])
conversion_table

In [ ]:
'is_relation_many_to_one' in conversion_table.keys()

In [ ]:
None or 'appel'

In [ ]:
import ast

dct = '{"doc.omstandigheden.keyword": ["<em>sloot</em>"], "doc.omstandigheden": ["<em>sloot</em>"]}'
dct = res = ast.literal_eval(dct)
lst = [item.replace("doc.", "") + ": " + ",".join(dct[item]) for item in dct.keys()]
"; ".join(lst)


In [15]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, connections, Q

client = Elasticsearch('http://elasticsearch:9200')
#connections.create_connection(hosts=['localhost'], timeout=20)

qq = {'simple_query_string': {'query': 'Kinderschoen', 'analyze_wildcard':True, "fields": ['*']}}


s = Search(using=client, index='def_artefact')
q= Q("simple_query_string", query='kind*', fields=['*'])
f= Q("multi_match", query='Leer', fields=['artefactsoort'])
s = s.query(q).highlight('*').filter(f)[0:1000]

response = s.execute()
print(response.hits.total.value)
#response.to_dict()

s.to_dict()

client.indices.exists(index="def_artefact")

64


True

In [8]:
from sqlalchemy import create_engine, inspect, MetaData

engine = create_engine(config.SQLALCHEMY_DATABASE_URI)    

metadata_obj = MetaData()
metadata_obj.reflect(bind=engine)
list(metadata_obj.tables.keys())



/tmp/ipykernel_97/2075969698.py:6: SAWarning: Did not recognize type 'geometry' of column 'location'
  metadata_obj.reflect(bind=engine)


['spatial_ref_sys',
 'Def_ABR',
 'alembic_version',
 'observation',
 'ab_user',
 'ab_register_user',
 'Def_Project',
 'Def_Stelling',
 'Def_Put',
 'ab_permission',
 'ab_permission_view',
 'ab_view_menu',
 'ab_user_role',
 'ab_role',
 'ab_permission_view_role',
 'Def_Spoor',
 'Def_Vlak',
 'Def_Monster',
 'Def_Doos',
 'Def_Vondst',
 'Def_Plaatsing',
 'Def_artefact_abr',
 'Def_Artefact',
 'Def_Standplaats',
 'Def_Partij',
 'Def_Bruikleen',
 'Def_Vindplaats',
 'Def_Bestand',
 'Def_Vulling',
 'Def_Monster_Botanie',
 'Def_Monster_Schelp',
 'topology',
 'layer']